# Temporal PageRank

In [1]:
import scipy.stats
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import operator
import copy
from datetime import datetime, timedelta
import os.path

### Beolvasás

In [2]:
def readRealGraph(filepath):
    edgesTS = []
    nodes = set()
    edges = set()
    lookup = {}          # szótár, kulcsok az élek lesznek, ezekhez rendel egy id-t
    c = 0
    with open(filepath,'r') as fd:
        for line in fd.readlines():

            line = line.strip()
            items = line.split(' ')
            tstamp = ' '.join(items[0:2])
            tstamp = tstamp[1:-1]
            tstamp = datetime.strptime(tstamp, '%Y-%m-%d %H:%M:%S')
            t = items[2:4]
            t = list(map(int,t))
            if t[0] == t[1]:
                continue
            #t.sort(); #undirected

            if tuple(t) in lookup.keys():
                num = lookup[tuple(t)]
            else:
                num = c
                lookup[tuple(t)] = c
                c += 1
            edgesTS.append((tstamp, tuple(t), num ))
            nodes.add(t[0])
            nodes.add(t[1])
            edges.add(tuple([t[0],t[1]]))
    fd.close()
    return edgesTS, nodes, edges


### gráf csinálás

In [8]:
def getSubgraph(G, N = 1000):
    Gcc = sorted(nx.connected_component_subgraphs(G.to_undirected()), key = len, reverse=True)
    print (len(Gcc))
    nodes = set()
    i = 0

    while len(nodes) < N:
        s = np.random.choice(Gcc[i].nodes())
        i += 1
        nodes.add(s)
        for edge in nx.bfs_edges(G.to_undirected(), s):
            nodes.add(edge[1])
            if len(nodes) == N:
                break
    return nx.subgraph(G, nodes)

In [9]:
def getGraph(edgesTS):
    G = nx.DiGraph()
    edges = {}

    for item in edgesTS:
        edge = item[1]
        edges[edge] = edges.get(edge, 0.0) + 1.0

    #nrm = float(sum(edges.values()))
    G.add_edges_from([(k[0],k[1], {'weight': v}) for k,v in edges.items()])
    #G.add_edges_from([tuple(edge)])
    return G


def weighted_DiGraph():
    
    edgesTS, _, _ = readRealGraph(os.path.join('.','..',"Data","facebook.txt"))
    G = getGraph(edgesTS)
    G = nx.DiGraph(G)
    G.remove_edges_from(G.selfloop_edges())
    G = getSubgraph(G, 100)
    
    for i in G.nodes_iter():
        if G.out_degree(i) == 0:
            for j in G.nodes_iter():
                if i != j:
                    G.add_edge(i, j, weight=1.0)

#    print (nx.info(G))

#    w = 1.0/G.number_of_edges()
#    for i in G.edges_iter():
#        G[i[0]][i[1]]['weight'] = w
        
    nrm = float(sum(G.out_degree(weight = 'weight').values()))
    for i in G.edges_iter(data=True):
        G[i[0]][i[1]]['weight'] = i[-1]['weight']/nrm
        
    return G

In [10]:
face_graph = weighted_DiGraph()

344


In [ ]:
# nx.pagerank(face_graph)

In [11]:
print (nx.info(face_graph))

Name: 
Type: DiGraph
Number of nodes: 100
Number of edges: 3419
Average in degree:  34.1900
Average out degree:  34.1900


### Temporal PR

In [6]:
def flowPR(p_prime_nodes, ref_pr, stream, RS, current, iters = 1000000, alpha = 0.85, beta=0.001, gamma=0.9999, normalization = 1.0, padding = 0):
    """ current: s
        RS: r
        p_prime_node: h*/h'
        """
    if beta == 1.0:
        beta = 0.0
        
    tau = []
    pearson = []
    spearman = []
    error = []
    x = []
    i = 0

    rank_order = [key for (key, value) in sorted(ref_pr.items(), key=operator.itemgetter(1), reverse=True)]
    ordered_pr = np.array([ref_pr[k] for k in rank_order])

    for e in stream:
        i += 1

        RS[e[0]] = RS.get(e[0], 0.0) * gamma + 1.0 * (1.0 - alpha) * p_prime_nodes[e[0]] * normalization
        RS[e[1]] = RS.get(e[1], 0.0) * gamma + (current.get(e[0], 0.0) + 1.0 * (1.0 - alpha) * p_prime_nodes[e[0]]) * alpha * normalization
        current[e[1]] = current.get(e[1], 0.0) + (current.get(e[0], 0.0) + 1.0 * (1.0 - alpha)* p_prime_nodes[e[0]]) * alpha *(1 - beta)
        current[e[0]] = current.get(e[0], 0.0) * beta


        if (i % 100 == 0 or i == len(stream)) and len(RS) == len(ordered_pr):
            if i == iters-1:
                print (sum(RS.values()))
            sorted_RS4 = np.array([RS[k] / sum(RS.values()) for k in rank_order])
            tau.append(scipy.stats.kendalltau(sorted_RS4, ordered_pr)[0])
            pearson.append(scipy.stats.pearsonr(sorted_RS4, ordered_pr)[0])
            spearman.append(scipy.stats.spearmanr(sorted_RS4, ordered_pr)[0])
            error.append(np.linalg.norm(sorted_RS4 - ordered_pr))
            x.append(i+padding)

        if i == iters-1:
            print (sum(RS.values()))

    sorted_RS4 = np.array([RS[k] / sum(RS.values()) for k in rank_order])

    return RS, current, tau, spearman, pearson, error, x


### Számolás egy 100 csúcsú részgráfra (facebook)

In [14]:
G = weighted_DiGraph()
norm = sum(G.out_degree(weight='weight').values())
sampling_edges = {e[:-1]: e[-1]['weight']/norm for e in G.edges_iter(data=True)}
stream = list(sampling_edges.keys())

# basic (degree personalization)
personalization = {k: v / norm for k, v in G.out_degree(weight='weight').items()}
p_prime_nodes = {i: personalization[i]/G.out_degree(i, weight='weight') for i in G.nodes_iter()}
pr_basic = nx.pagerank(G, personalization=personalization, weight='weight')
RS4_basic, current_basic = {}, {}
RS4_basic, current_basic, tau_basic, spearman_basic, pearson_basic, error_basic, x = flowPR(p_prime_nodes, pr_basic, stream, RS4_basic, current_basic)


344


In [15]:
RS4_basic, current_basic, tau_basic, spearman_basic, pearson_basic, error_basic, x

({93: 4.442844420564688,
  307: 3.8794925017251156,
  309: 5.506031621591319,
  313: 23.532798873636633,
  315: 22.36274351303042,
  532: 18.27154640794243,
  1045: 5.60573737877283,
  1219: 5.521092207327903,
  1660: 4.52975125119724,
  1897: 6.6316670827576525,
  1898: 8.406877953863116,
  1904: 47.82289830041694,
  1908: 8.75631877593844,
  1914: 28.716720217680322,
  2323: 5.462412042793278,
  2436: 27.107954674962865,
  2476: 21.93851598173969,
  3371: 8.94151151837056,
  3374: 17.902244246911895,
  3609: 5.878176131555866,
  3701: 13.120749235973088,
  3870: 24.76324255754811,
  3872: 9.1838211184592,
  3879: 29.05304104216796,
  3880: 5.960781953128719,
  3899: 5.673360477929422,
  3901: 12.545935942552454,
  3904: 19.45514611725013,
  3905: 10.117000648073926,
  3908: 16.235858126765912,
  3920: 6.5749417058400645,
  3980: 4.526761507878681,
  3986: 18.606032435170185,
  4960: 7.915647119839862,
  5055: 8.702264127373207,
  5075: 4.218091872817256,
  5093: 18.921373314901555,
 